   0000_16_230115_142_subway_total_map
        
   going deeper node 3-8   
    * 전국 도시철도 역사정보 표준데이터 활용 

    data: subway_total_3.csv
        save: k_station_df.csv
        display : subway.html

In [1]:
# 필요한 라이브러리 import 

import requests
import pprint
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

print("슝~")

슝~


In [2]:
#9 Folium를 활용한 공간 데이터 시각화
### 전국도시철도역사정보표준데이터 가져오기
                                                
subway_station_info_df= pd.read_csv('c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/001_map_view_real/02_map_subway/subway_total_3.csv')
#subway_station_info_df = pd.read_csv(os.getenv('HOME') + "/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/004_node_data/train_station_info.csv", encoding="cp949")
subway_station_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
0,코레일,1호선,가능,경기도 의정부시 가능동 197-1,경기도 의정부시 평화로 633,959850.319275,1.972172e+06,127.044255,37.748305
1,코레일,1호선,가산디지털단지,서울특별시 금천구 가산동 468-4,서울특별시 금천구 벚꽃로 309,945373.187085,1.942741e+06,126.882129,37.482290
2,코레일,1호선,간석,인천광역시 남동구 간석4동 762번지,인천광역시 남동구 석정로 522-14,928718.830404,1.940941e+06,126.693945,37.464929
3,코레일,1호선,개봉,서울특별시 구로구 개보동 415,서울특별시 구로구 경인로40길 47,943348.199060,1.944111e+06,126.859121,37.494522
4,코레일,1호선,관악,경기도 안양시 만악구 석수동 101-16,경기도 안양시 만안구 경수대로1273번길 46,947714.862265,1.935684e+06,126.909115,37.418821


In [3]:
subway_station_info_df.tail()

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
755,인천교통공사,인천2호선,인천가좌,인천광역시 서구 가좌동 110,인천광역시 서구 가석로 67,927092.229580,1.943770e+06,126.675273,37.490308
756,인천교통공사,인천2호선,인천대공원,인천광역시 남동구 장수동 606-9,인천광역시 남동구 수인로 3677,933899.201155,1.939079e+06,126.752688,37.448535
757,인천교통공사,인천2호선,인천시청,인천광역시 남동구 간석동 927,인천광역시 남동구 구월로 99,929361.809352,1.940042e+06,126.701302,37.456881
758,인천교통공사,인천2호선,주안,인천광역시 미추홀구 주안동 125,인천광역시 미추홀구 주안로 91-1,927471.607452,1.940931e+06,126.679844,37.464748
759,인천교통공사,인천2호선,주안국가산단,인천광역시 서구 가좌동 606-11,인천광역시 서구 방축로 343,927603.904677,1.941968e+06,126.681238,37.474101


In [4]:
## 위도(lat), 경도(lng)의 데이터 타입을 float으로 변경하기
subway_station_info_df["lat"] = subway_station_info_df["lat"].astype(float)
subway_station_info_df["lng"] = subway_station_info_df["lng"].astype(float)

subway_station_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   철도운영기관명  760 non-null    object 
 1   선명       760 non-null    object 
 2   역명       760 non-null    object 
 3   지번주소     717 non-null    object 
 4   도로명주소    760 non-null    object 
 5   x        760 non-null    float64
 6   y        760 non-null    float64
 7   lng      760 non-null    float64
 8   lat      760 non-null    float64
dtypes: float64(4), object(5)
memory usage: 53.6+ KB


In [5]:
subway_station_info_df[subway_station_info_df['lng'].isnull()]

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat


In [6]:
# # subway_station_info_df=subway_station_info_df.drop([498], axis = 0,inplace=False)  
# subway_station_info_df=subway_station_info_df.drop([517], axis = 0,inplace=False)  
# subway_station_info_df=subway_station_info_df.drop([623], axis = 0,inplace=False)  
# subway_station_info_df=subway_station_info_df.drop([701], axis = 0,inplace=False)  

In [7]:
# subway_station_info_df = subway_station_info_df.dropna(axis=0)

In [8]:
subway_station_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   철도운영기관명  760 non-null    object 
 1   선명       760 non-null    object 
 2   역명       760 non-null    object 
 3   지번주소     717 non-null    object 
 4   도로명주소    760 non-null    object 
 5   x        760 non-null    float64
 6   y        760 non-null    float64
 7   lng      760 non-null    float64
 8   lat      760 non-null    float64
dtypes: float64(4), object(5)
memory usage: 53.6+ KB


In [9]:
# Folium 초기 Map 객체 생성
# 초기 Map 메소드에는 초기 중심값 좌표가 들어가야 합니다.  
base_map = folium.Map(
    location=[37.566345, 126.977893]
)


In [10]:
# 지도에 Circle 찍기
for index_num in range(len(subway_station_info_df)):
 
    folium.Circle(
        location=[subway_station_info_df.loc[index_num, "lat"], subway_station_info_df.loc[index_num, "lng"]],
        radius=200,
        color="blue",
        popup=subway_station_info_df.loc[index_num, "역명"]
    ).add_to(base_map)

base_map

In [11]:
base_map.save("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/subway.html")

In [12]:
subway_station_info_df.to_csv("k_station_df.csv", index = False)